#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter


In [9]:
loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [10]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [15]:
embeddings = OllamaEmbeddings()

In [16]:
db=FAISS.from_documents(docs,embeddings)
db

In [17]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [18]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [19]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='685e1469-e073-4d94-8866-c1e9a3a243a3', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  15381.061),
 (Document(id='584b6f2e-3fe7-415e-a3be-c1898dde6b37', metadata={'source': 'speech.txt'},

In [20]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[1.888381838798523,
 -3.116117238998413,
 1.9619596004486084,
 1.4583569765090942,
 -0.9296711087226868,
 0.6837998032569885,
 1.5219963788986206,
 -0.552751898765564,
 0.8918653130531311,
 -2.013533115386963,
 1.292631983757019,
 -1.959432601928711,
 -0.6345752477645874,
 1.534688115119934,
 -0.4709117114543915,
 -2.040411949157715,
 -0.4415338933467865,
 0.03531895950436592,
 0.7380772829055786,
 -1.8991410732269287,
 -0.6681324243545532,
 -0.9819162487983704,
 2.177724599838257,
 -1.9868284463882446,
 0.7717866897583008,
 -0.9921314716339111,
 0.333592027425766,
 -1.6840544939041138,
 0.2087569683790207,
 -0.8434448838233948,
 2.0993359088897705,
 -2.793506145477295,
 -2.6978442668914795,
 3.728616952896118,
 2.318737030029297,
 -5.091105937957764,
 -1.0624850988388062,
 1.3555043935775757,
 -1.2416011095046997,
 -1.3923412561416626,
 -0.49567511677742004,
 -2.045727491378784,
 2.1134815216064453,
 0.37947404384613037,
 0.9773537516593933,
 0.5601382851600647,
 -0.20688942074775696,

In [21]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='685e1469-e073-4d94-8866-c1e9a3a243a3', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='584b6f2e-3fe7-415e-a3be-c1898dde6b37', metadata={'source': 'speech.txt'}, page_content='â

In [22]:
### Saving And Loading
db.save_local("faiss_index")

In [23]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [24]:
docs

[Document(id='685e1469-e073-4d94-8866-c1e9a3a243a3', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='584b6f2e-3fe7-415e-a3be-c1898dde6b37', metadata={'source': 'speech.txt'}, page_content='â